# Structured Logging Demo

In [1]:
import sqlite3
from importlib import import_module
from app.factories.logging_provider import LoggingProvider
from app.extensions.context_providers.dummy_context_provider import DummyContextProvider
from app.extensions.tool_providers.dummy_tool_provider import DummyToolProvider
from app.extensions.agents.dummy_agent import DummyAgent
from app.extensions.experiment_provider import ExperimentProvider


In [2]:
conn = sqlite3.connect(':memory:')
logger = LoggingProvider(connection=conn)
ctx = DummyContextProvider(logger=logger)
tool = DummyToolProvider(logger=logger)
agent = DummyAgent(logger=logger)
exp = ExperimentProvider('demo', 'demo experiment', 'test', 'v1', 1, 0.0, 'engine', 'eval', '1.0', logger=logger)
ctx.get_context(experiment_id='demo', round=0)
tool.run(experiment_id='demo', round=0)
agent.run(experiment_id='demo', round=0)
exp.complete(1.0, True, 'done')


In [3]:
cur = conn.cursor()
tables = [
        'context_retrieval_log',
        'tool_invocation_log',
        'agent_action_log',
        'system_event_log',
        'experiment_log'
    ]
counts = {t: cur.execute(f'SELECT COUNT(*) FROM {t}').fetchone()[0] for t in tables}
print(counts)
